<a href="https://colab.research.google.com/github/joekelly211/masfi/blob/main/7_uncertainty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and directories

In [ ]:
# Define base directory
# Use '/content/drive/MyDrive/' for a personal drive
# Use '/gdrive/Shareddrives/' for a shared drive (must be created first)

base_dir = "/gdrive/Shareddrives/masfi"
# base_dir = '/content/drive/MyDrive/masfi'

# Mount Google Drive
from google.colab import drive
import os
import sys
if base_dir.startswith('/gdrive/Shareddrives/'):
  drive.mount('/gdrive', force_remount=True)
elif base_dir.startswith('/content/drive/MyDrive/'):
  drive.mount('/content/drive', force_remount=True)
  os.makedirs(base_dir, exist_ok=True)
else: print("Create a base_dir beginning with '/gdrive/Shareddrives/' or '/content/drive/MyDrive/'.")

_path_to_add = os.path.realpath(base_dir)
if _path_to_add not in sys.path:
    sys.path.append(_path_to_add)

In [ ]:
# Capture outputs
%%capture
# Installs and upgrades
!pip install geopandas
!pip install xgboost

In [ ]:
# Imports
try: import cupy # Only works on GPU runtime
except: None
import ast
import gc
import geopandas as gpd
from google.colab import runtime
import ipywidgets as widgets
import json
import numpy as np
from numpy import random
from numpy.random import normal
from os import makedirs
from os.path import join, exists, basename
from osgeo import gdal
gdal.UseExceptions()
import pickle
import pandas as pd
import scipy.stats as st
import shutil
from shutil import copyfile
import xgboost as xgb
import warnings

In [ ]:
# Define directories
areas_dir = join(base_dir, "1_areas")
polygons_dir = join(areas_dir, "polygons")
feature_dir = join(base_dir, "3_features")
feature_final_dir = join(feature_dir, "final")
models_dir = join(base_dir, "5_models")
scenarios_dir = join(base_dir, "6_scenarios")
masks_dir = join(scenarios_dir, "scenario_masks")
uncertainty_dir = join(base_dir, "7_uncertainty")

# Create directories
makedirs(uncertainty_dir, exist_ok=True)

In [ ]:
# Global function: export an array as a .tif
template_tif_path = join(areas_dir, "template.tif")
nodatavalue = -11111
compress = True
def export_array_as_tif(input_array, output_tif, template=template_tif_path, nodatavalue=nodatavalue, compress=compress, dtype=gdal.GDT_Float32):
    template_ds = gdal.Open(template)
    template_band = template_ds.GetRasterBand(1)
    template_dimensions, template_projection = template_ds.GetGeoTransform(), template_ds.GetProjection()
    if compress: options = ['COMPRESS=ZSTD', 'ZSTD_LEVEL=1'] # Good speed / size ratio
    else: options = []
    if input_array.dtype == 'int16': dtype = gdal.GDT_Int16
    driver = gdal.GetDriverByName("GTiff").Create(output_tif, template_band.XSize, template_band.YSize, 1, dtype, options=options)
    driver.GetRasterBand(1).WriteArray(input_array)
    driver.GetRasterBand(1).SetNoDataValue(nodatavalue)
    driver.SetGeoTransform(template_dimensions)
    driver.SetProjection(template_projection)
    template_ds = driver = None

# Global function: read raster as array
def read_raster_as_array(path):
    ds = gdal.Open(path)
    arr = ds.ReadAsArray()
    ds = None
    return arr

# Select model

In [ ]:
# Select a baseline model, tested and trained in advance.
# The target must have an uncertainty metric - otherwise
# skip to the next notebook '8_statistics' and use the outputs
# of the '6_scenarios' notebook.
model_exists = False
for subdir, dirs, files in os.walk(models_dir):
  for file in files:
    if file == 'model.json':
      print(f'selected_model = "{subdir.split(f"{models_dir}/",1)[1]}"')
      model_exists = True
if not model_exists:
  print("No model exists.")

In [ ]:
selected_model = "agbd_251203_161707"

# This must be True when using AlphaEarth features.
alpha_earth = False

# Define model directories
selected_model_dir = join(models_dir,selected_model)
selected_model_json = join(selected_model_dir, "model.json")
selected_model_descr_dir = join(selected_model_dir, "model_description.json")
selected_model_dataset_path = join(selected_model_dir, f"{selected_model}.pkl")
selected_model_dataset = pd.read_pickle(selected_model_dataset_path)

# Read description for model dataset attributes
with open(join(selected_model_dir,"model_dataset_description.json")) as model_dataset_description_json:
  model_dataset_description = json.load(model_dataset_description_json)
model_dataset_name = model_dataset_description["model_dataset_name"]
number_of_columns = model_dataset_description["number_of_columns"]
number_of_rows = model_dataset_description["number_of_rows"]
id_column = model_dataset_description["id_column"]
selected_target = model_dataset_description["selected_target"]
uncertainty = model_dataset_description["uncertainty"]
covariates_renamed = model_dataset_description["covariates_renamed"]
selected_features = model_dataset_description["selected_features"] + model_dataset_description["covariates_renamed"]
categorical_features_mappings = model_dataset_description["categorical_features_mappings"]
categorical_columns = model_dataset_description["categorical_columns"]
descriptive_parameters = model_dataset_description["descriptive_parameters"]
sample_imported_dataset = model_dataset_description["sample_imported_dataset"]
sample_imported_dataset_by_percent = model_dataset_description["sample_imported_dataset_by_percent"]
sample_imported_dataset_value = model_dataset_description["sample_imported_dataset_value"]

# Reload hyperparameters
with open(selected_model_descr_dir) as model_description_json:
  model_description = json.load(model_description_json)
final_hyperparameters = ast.literal_eval(model_description["hyperparameters"])

# Remove early stopping and replace with mean n_estimators
if "early_stopping_rounds" in final_hyperparameters:
  final_hyperparameters = {k:v for k, v in final_hyperparameters.items() if k != "early_stopping_rounds"}
  final_hyperparameters["n_estimators"] = round(model_description["n_estimators mean"])

# Define directories
scenarios_model_dir = join(scenarios_dir,selected_model)
features_dir = join(scenarios_model_dir, "features")
tile_templates_dir = join(scenarios_model_dir, 'tile_templates')
tile_feature_stacks_dir = join(scenarios_model_dir, 'tile_feature_stacks')
uncertainty_selected_model_dir = join(uncertainty_dir, selected_model)
model_iterations_dir = join(uncertainty_selected_model_dir, "model_iterations")
scenario_iterations_dir = join(uncertainty_selected_model_dir, "scenario_iterations")
tile_prediction_cache_dir = join(uncertainty_selected_model_dir, "tile_prediction_cache")
predictions_dir = join(uncertainty_selected_model_dir, "uncertainty_predictions")

# Create directories
makedirs(uncertainty_selected_model_dir, exist_ok=True)
makedirs(model_iterations_dir, exist_ok=True)
makedirs(scenario_iterations_dir, exist_ok=True)
makedirs(tile_prediction_cache_dir, exist_ok=True)
makedirs(predictions_dir, exist_ok=True)

# Model iterations

In [ ]:
# Verify that the target is equal to the mean
print(f'mean = "{selected_target}"')

# Calculate se from columns flagged 'uncertainty'
if len(uncertainty)==0:
  print("There are no flagged uncertainty columns to calculate SE from.")
  print("Manually create the metric from the available columns.")
  for col in selected_model_dataset.columns:
    print(f"{col}")
else:
  for col in selected_model_dataset.columns:
    if col in uncertainty and col not in selected_target:
      print(f'se = "{col}"')

In [ ]:
mean = "tar_agbd"
se = "tar_agbd_se"

# GEDI L4A agbd_se represents the prediction standard error incorporating both model
# parameter uncertainty and residual variance (GEDI L4A ATBD Eq. 9, Kellner et al. 2021).
# For Monte Carlo uncertainty propagation, agbd_se directly parameterises the standard
# deviation of the prediction error distribution for each footprint.

# Set model iterations
model_iterations = 10

# Define model (y axis changes for each iteration based on mean and se arrays)
model_dataset_x = selected_model_dataset[selected_features].copy()
for col in categorical_columns:
    if col in model_dataset_x.columns:
        model_dataset_x[col] = model_dataset_x[col].astype('category')
mean_array = selected_model_dataset[mean].values
se_array = selected_model_dataset[se].values

# Detect model type from existing model or determine from target
existing_model_path = join(model_iterations_dir, "model_iteration_1.json")
if exists(existing_model_path):
    # Load existing model and detect type
    temp_booster = xgb.Booster()
    temp_booster.load_model(existing_model_path)
    model_config = json.loads(temp_booster.save_config())

    objective_name = model_config['learner']['objective']['name']
    num_class = int(model_config['learner']['learner_model_param'].get('num_class', '0'))
    classification = any(obj_type in objective_name for obj_type in ['logistic', 'softprob', 'softmax'])
    multiclass = classification and num_class > 2
else:
    # Determine from target variable characteristics
    unique_values = len(np.unique(mean_array))
    if unique_values <= 10 and all(val == int(val) for val in np.unique(mean_array)):
        classification = True
        multiclass = unique_values > 2
        num_class = unique_values if multiclass else 0
    else:
        classification = False
        multiclass = False
        num_class = 0

# Set model type
if classification:
    XGBPredictor = xgb.XGBClassifier(**final_hyperparameters)
    if multiclass: print(f"Model type: Multiclass classification ({num_class} classes)")
    else: print("Model type: Binary classification")
else:
    XGBPredictor = xgb.XGBRegressor(**final_hyperparameters)
    print("Model type: Regression")

model_params = XGBPredictor.get_params()
model_params['eval_metric'] = model_description['metric_used_for_training']
# Default fix for new XGBoost version
[model_params.pop(key, None) for key in ['n_estimators', 'enable_categorical', 'missing']]

# Progress label
model_progress_index = 0
model_progress_label = widgets.Label(f"Model iteration: {model_progress_index}/{model_iterations}")
display(model_progress_label)

for model_iteration in range(1,model_iterations+1):
  # Set model iteration filename and check if already exists
  model_iteration_filename = f"model_iteration_{model_iteration}.json"
  model_iteration_path = join(model_iterations_dir, model_iteration_filename)
  # If model iteration does not exist...
  if not exists(model_iteration_path):
    # Set the random seed based on iteration for replicability
    np.random.seed(model_iteration)
    # Set a normal distribution sample as the y for this iteration
    model_dataset_y = normal(mean_array, se_array)
    # Create DMatrix objects
    model_dtrain = xgb.DMatrix(model_dataset_x, model_dataset_y, enable_categorical=True)
    # Train the model iteration using the tested hyperparameters
    model = xgb.train(model_params,
                        model_dtrain,
                        num_boost_round=final_hyperparameters['n_estimators'],
                        verbose_eval=True)
    # Save the model iteration
    model.save_model(model_iteration_path)
  # Update progress
  model_progress_index += 1
  model_progress_label.value = f"Model iteration: {model_progress_index}/{model_iterations}"
print("All model iterations have been trained and saved.")

# Scenario iterations

In [ ]:
# Scenarios must be designed and tested using 06_scenarios first.

# Check existing tile parameters
template_tile_list = [f for f in os.listdir(tile_templates_dir) if f.endswith('.tif') and f.startswith('template_tile')]
n_tiles = len(template_tile_list)
assert n_tiles > 0, "There are no template tiles. Run the template tiles section, even if only one is created."
if n_tiles == 1: print(f"# There is 1 template tile.\n")
if n_tiles > 1: print(f"# There are {n_tiles} template tiles.\n")

# Collect available scenarios from the feature stack tiles directory
scenario_stacks_list = []
for scenario in os.listdir(tile_feature_stacks_dir):
    stack_files = [f for f in os.listdir(join(tile_feature_stacks_dir, scenario)) if f.startswith('feature_stack_')]
    if len(stack_files) == n_tiles:
        scenario_stacks_list.append(scenario)

print("scenarios_to_predict = [")
for scenario in sorted(scenario_stacks_list):
  print(f'  "{scenario}",')
print("]")

In [ ]:
# There is 1 template tile.

scenarios_to_predict = [
  "2018",
  "2019",
  "2020",
  "2021",
  "2021_undisturbed_since_1993",
  "2021_undisturbed_since_oldgrowth_1",
  "2021_undisturbed_since_oldgrowth_2",
  "2021_oldgrowth_recovery_1",
  "2021_oldgrowth_recovery_2",
  "2022",
  "2023",
  "2024",
  "2024_undisturbed_since_1996",
  "2024_undisturbed_since_1997",
  "2024_undisturbed_since_1998",
  "2024_undisturbed_since_1999",
  "2024_undisturbed_since_2000",
  "2024_undisturbed_since_2001",
  "2024_undisturbed_since_2002",
  "2024_undisturbed_since_2003",
  "2024_undisturbed_since_2004",
  "2024_undisturbed_since_2005",
  "2024_undisturbed_since_2006",
  "2024_undisturbed_since_2007",
  "2024_undisturbed_since_2008",
  "2024_undisturbed_since_2009",
  "2024_undisturbed_since_2010",
  "2024_undisturbed_since_2011",
  "2024_undisturbed_since_2012",
  "2024_undisturbed_since_2013",
  "2024_undisturbed_since_2014",
  "2024_undisturbed_since_2015",
  "2024_undisturbed_since_2016",
  "2024_undisturbed_since_2017",
  "2024_undisturbed_since_2018",
  "2024_undisturbed_since_2019",
  "2024_undisturbed_since_2020",
  "2024_undisturbed_since_2021",
  "2024_undisturbed_since_2022",
  "2024_undisturbed_since_2023",
  "2024_undisturbed_since_2024",
  "2024_undisturbed_since_oldgrowth_1",
  "2024_undisturbed_since_oldgrowth_2",
  "2024_oldgrowth_recovery_1",
  "2024_oldgrowth_recovery_2",
  "2024_road_mat_daling_deforestation_2023_30m_degradation_buffer",
]

In [ ]:
# Set the number of scenario iterations. It must be <= the number of model iterations available.
scenario_iterations = 10

# Assert the number of scenario iterations is <= the number of model iterations available.
model_iterations_available = len([f for f in os.listdir(model_iterations_dir) if f.startswith('model_iteration_') and f.endswith('.json')])
assert scenario_iterations <= model_iterations_available, f"Reduce the number of scenario iterations to <= {model_iterations_available}."

# Prediction raster precision. GEDI AGBD can be set to 0, any higher is
# spurious due to the wide prediction intervals of the source data.
raster_precision = 0

# Probabilities instead of classes IF binary classification
predict_probabilities = False

# Classification threshold IF binary classification
classification_threshold = 0.5

# Detect GPU availability and set predictor type
try:
    test_array = cupy.array([1, 2, 3])
    del test_array
    predictor_type = 'gpu_predictor'
    gpu_id, use_gpu = 0, True
    print("GPU detected and accessible - using GPU to load the feature stack and to predict.")
except:
    predictor_type = 'cpu_predictor'
    gpu_id, use_gpu = -1, False
    print("GPU not accessible - using CPU prediction")
xgb.set_config(verbosity=0, use_rmm=use_gpu)

# Detect model type using first model iteration
first_model_path = join(model_iterations_dir, "model_iteration_1.json")
booster = xgb.Booster()
booster.load_model(first_model_path)
model_config = json.loads(booster.save_config())
objective_name = model_config['learner']['objective']['name']
num_class = int(model_config['learner']['learner_model_param'].get('num_class', '0'))
classification = any(obj_type in objective_name for obj_type in ['logistic', 'softprob', 'softmax'])
multiclass = classification and num_class > 2
if classification and multiclass: print(f"Model type: Multiclass classification ({num_class} classes)")
elif classification: print("Model type: Binary classification")
else: print("Model type: Regression")
booster = None

# Check existing tile parameters
template_tile_list = [f for f in os.listdir(tile_templates_dir) if f.endswith('.tif') and f.startswith('template_tile')]
n_tiles = len(template_tile_list)
if n_tiles < 1: print("There are currently no template tiles.")
else:
  template_tile = gdal.Open(join(tile_templates_dir, 'template_tile_1.tif'))
  template_tile_x = template_tile.GetRasterBand(1).XSize
  template_tile = None
  print(f"There are {n_tiles} template tiles.")

if alpha_earth: template_base_path = next(r for r in model_scenario_features_dirs if all(c not in r for c in covariates))
else: template_base_path = template_tif_path

# Progress labels
n_scenarios = len(scenarios_to_predict)
scenario_progress_label = widgets.Label(value=f"Scenario progress: 0 / {n_scenarios}")
tile_progress_label = widgets.Label(value="Tile progress: -")
iteration_progress_label = widgets.Label(value="Iteration progress: -")
display(scenario_progress_label, tile_progress_label, iteration_progress_label)

# Loop through each scenario
for scenario_index, scenario in enumerate(scenarios_to_predict):
  scenario_feature_stack_dir = join(tile_feature_stacks_dir, scenario)
  # Create scenario iterations directory
  iterations_dir = join(scenario_iterations_dir, f"{scenario}_iterations")
  makedirs(iterations_dir, exist_ok=True)

  # Check if all scenario iterations already exist
  scenario_iteration_list = []
  for model_iteration in range(1, scenario_iterations + 1):
    prediction_iteration_filename = f"{scenario}__{selected_model}_iteration_{model_iteration}.tif"
    prediction_iteration_path = join(iterations_dir, prediction_iteration_filename)
    scenario_iteration_list.append(prediction_iteration_path)
  all_scenario_iterations_exist = all(exists(p) for p in scenario_iteration_list)

  if not all_scenario_iterations_exist:
    n_stacks = len([f for f in os.listdir(scenario_feature_stack_dir) if f.startswith('feature_stack_')])

    # Single-stack prediction
    if n_stacks == 1:
      tile_progress_label.value = "Tile progress: 0 / 1"
      iteration_progress_label.value = f"Iteration progress: 0 / {scenario_iterations}"

      # Load template parameters
      template_tile_path = join(tile_templates_dir, "template_tile_1.tif")
      template_tile = gdal.Open(template_tile_path)
      template_tile_y = template_tile.GetRasterBand(1).YSize
      template_tile_x = template_tile.GetRasterBand(1).XSize
      template_tile = None
      n_pixels = template_tile_y * template_tile_x
      # Load feature stack and valid indices
      stack_path = join(scenario_feature_stack_dir, f"feature_stack_{scenario}_1.npy")
      indices_path = join(scenario_feature_stack_dir, f"valid_indices_{scenario}_1.npy")
      feature_stack = np.load(stack_path)
      valid_indices = np.load(indices_path)
      n_valid = len(valid_indices)
      # Load to GPU if available and valid pixels exist
      if n_valid > 0 and use_gpu:
        try:
          feature_stack = cupy.asarray(feature_stack)
        except Exception as e:
          if "Memory allocation error" in str(e) or "Out of memory" in str(e):
            print("GPU memory insufficient, switching to CPU.")
            cupy.get_default_memory_pool().free_all_blocks()
            gc.collect()
            use_gpu = False
            predictor_type = 'cpu_predictor'
          else: raise

      # Predict all iterations using loaded stack
      iteration_progress_index = 0
      for model_iteration in range(1, scenario_iterations + 1):
        model_path = join(model_iterations_dir, f"model_iteration_{model_iteration}.json")
        prediction_iteration_filename = f"{scenario}__{selected_model}_iteration_{model_iteration}.tif"
        prediction_iteration_path = join(iterations_dir, prediction_iteration_filename)

        if not exists(prediction_iteration_path):
          # Load model iteration
          if classification:
            XGBPredictor = xgb.XGBClassifier()
            XGBPredictor.load_model(model_path)
            XGBPredictor.set_params(predictor=predictor_type)
            if use_gpu: XGBPredictor.set_params(device='cuda:0')
          else:
            XGBPredictor = xgb.XGBRegressor()
            XGBPredictor.load_model(model_path)
            XGBPredictor.set_params(predictor=predictor_type)
            if use_gpu: XGBPredictor.set_params(device='cuda:0')

          # Handle empty tiles
          if n_valid == 0:
            if raster_precision == 0:
              prediction_array = np.full((template_tile_y, template_tile_x), nodatavalue, dtype=np.int16)
            else:
              prediction_array = np.full((template_tile_y, template_tile_x), nodatavalue, dtype=np.float32)
          else:
            # Predict - terminate runtime if GPU prediction fails
            try:
              if classification and predict_probabilities and not multiclass:
                prediction_proba = XGBPredictor.predict_proba(feature_stack)
                prediction = prediction_proba[:, 1]
              else:
                if classification and not multiclass:
                  prediction_proba = XGBPredictor.predict_proba(feature_stack)
                  prediction = (prediction_proba[:, 1] > classification_threshold).astype(int)
                else:
                  prediction = XGBPredictor.predict(feature_stack)
                  if classification:
                    if prediction.ndim > 1 and prediction.shape[1] > 1: prediction = np.argmax(prediction, axis=1)
                    prediction = prediction.astype(int)
            except Exception as e:
              if "Memory allocation error" in str(e) or "Out of memory" in str(e):
                print("GPU memory insufficient for prediction. Terminating runtime to save compute units, restart with TPU.")
                runtime.unassign()
              else: raise
            # Reconstruct full array from valid indices (C-order)
            if raster_precision == 0:
              prediction_flat = np.full(n_pixels, nodatavalue, dtype=np.int16)
              prediction_flat[valid_indices] = np.round(prediction).astype(np.int16)
            else:
              prediction_flat = np.full(n_pixels, nodatavalue, dtype=np.float32)
              prediction_flat[valid_indices] = np.round(prediction, raster_precision)
            prediction = None
            prediction_array = prediction_flat.reshape((template_tile_y, template_tile_x), order='C')
            prediction_flat = None
          export_array_as_tif(prediction_array, prediction_iteration_path, template=template_base_path, compress=True)
          prediction_array = None

        iteration_progress_index += 1
        iteration_progress_label.value = f"Iteration progress: {iteration_progress_index} / {scenario_iterations}"
      # Clean up single-stack feature stack from memory
      feature_stack = valid_indices = None
      tile_progress_label.value = "Tile progress: 1 / 1"

    # Tiled prediction - load each stack once, predict all iterations per tile
    if n_stacks > 1:
      # Create tile cache directories for all iterations
      tile_cache_dirs = {}
      for model_iteration in range(1, scenario_iterations + 1):
        prediction_iteration_filename = f"{scenario}__{selected_model}_iteration_{model_iteration}.tif"
        tile_cache_iteration_dir = join(tile_prediction_cache_dir, prediction_iteration_filename[:-4])
        makedirs(tile_cache_iteration_dir, exist_ok=True)
        tile_cache_dirs[model_iteration] = tile_cache_iteration_dir

      # Process each tile: load stack once, predict all iterations
      for tile_count in range(1, n_stacks + 1):
        # Determine which iterations still need this tile
        iterations_needing_tile = []
        for model_iteration in range(1, scenario_iterations + 1):
          iteration_tile_path = join(tile_cache_dirs[model_iteration], f"scenario_tile_{tile_count}.tif")
          prediction_iteration_path = join(iterations_dir, f"{scenario}__{selected_model}_iteration_{model_iteration}.tif")
          if not exists(prediction_iteration_path) and not exists(iteration_tile_path):
            iterations_needing_tile.append(model_iteration)

        # Skip tile if no iterations need it
        n_iterations_for_tile = len(iterations_needing_tile)
        if n_iterations_for_tile == 0:
          iteration_progress_label.value = f"Iteration progress: {scenario_iterations} / {scenario_iterations}"
          tile_progress_label.value = f"Tile progress: {tile_count} / {n_stacks}"
          continue
        iteration_progress_label.value = f"Iteration progress: 0 / {n_iterations_for_tile}"

        # Load template tile parameters
        template_tile_path = join(tile_templates_dir, f"template_tile_{tile_count}.tif")
        template_tile = gdal.Open(template_tile_path)
        template_tile_y = template_tile.GetRasterBand(1).YSize
        template_tile_x = template_tile.GetRasterBand(1).XSize
        template_tile = None
        n_pixels = template_tile_y * template_tile_x

        # Load feature stack and valid indices once per tile
        stack_path = join(scenario_feature_stack_dir, f"feature_stack_{scenario}_{tile_count}.npy")
        indices_path = join(scenario_feature_stack_dir, f"valid_indices_{scenario}_{tile_count}.npy")
        if not exists(stack_path):
          print(f"Warning: {basename(stack_path)} not found. Skipping tile {tile_count}.")
          continue
        feature_stack = np.load(stack_path)
        valid_indices = np.load(indices_path)
        n_valid = len(valid_indices)

        # Handle empty tiles for all needed iterations
        if n_valid == 0:
          if raster_precision == 0:
            prediction_tile = np.full((template_tile_y, template_tile_x), nodatavalue, dtype=np.int16)
          else: prediction_tile = np.full((template_tile_y, template_tile_x), nodatavalue, dtype=np.float32)
          tile_iteration_index = 0
          for model_iteration in iterations_needing_tile:
            iteration_tile_path = join(tile_cache_dirs[model_iteration], f"scenario_tile_{tile_count}.tif")
            export_array_as_tif(prediction_tile, iteration_tile_path, template=template_tile_path, compress=False)
            tile_iteration_index += 1
            iteration_progress_label.value = f"Iteration progress: {tile_iteration_index} / {n_iterations_for_tile}"
          prediction_tile = None
          tile_progress_label.value = f"Tile progress: {tile_count} / {n_stacks}"
          continue

        # Load to GPU if available
        tile_use_gpu = use_gpu
        if use_gpu:
          try: feature_stack = cupy.asarray(feature_stack)
          except Exception as e:
            if "Memory allocation error" in str(e) or "Out of memory" in str(e):
              print(f"GPU memory insufficient for tile {tile_count}, using CPU.")
              cupy.get_default_memory_pool().free_all_blocks()
              gc.collect()
              tile_use_gpu = False
            else: raise

        # Predict all needed iterations for this tile
        tile_iteration_index = 0
        for model_iteration in iterations_needing_tile:
          iteration_tile_path = join(tile_cache_dirs[model_iteration], f"scenario_tile_{tile_count}.tif")
          model_path = join(model_iterations_dir, f"model_iteration_{model_iteration}.json")
          # Load model iteration
          if classification:
            XGBPredictor = xgb.XGBClassifier()
            XGBPredictor.load_model(model_path)
            XGBPredictor.set_params(predictor='gpu_predictor' if tile_use_gpu else 'cpu_predictor')
            if tile_use_gpu: XGBPredictor.set_params(device='cuda:0')
          else:
            XGBPredictor = xgb.XGBRegressor()
            XGBPredictor.load_model(model_path)
            XGBPredictor.set_params(predictor='gpu_predictor' if tile_use_gpu else 'cpu_predictor')
            if tile_use_gpu: XGBPredictor.set_params(device='cuda:0')
          # Predict - terminate runtime if GPU prediction fails
          try:
            if classification and predict_probabilities and not multiclass:
              prediction_proba = XGBPredictor.predict_proba(feature_stack)
              prediction = prediction_proba[:, 1]
            else:
              if classification and not multiclass:
                prediction_proba = XGBPredictor.predict_proba(feature_stack)
                prediction = (prediction_proba[:, 1] > classification_threshold).astype(int)
              else:
                prediction = XGBPredictor.predict(feature_stack)
                if classification:
                  if prediction.ndim > 1 and prediction.shape[1] > 1: prediction = np.argmax(prediction, axis=1)
                  prediction = prediction.astype(int)
          except Exception as e:
            if "Memory allocation error" in str(e) or "Out of memory" in str(e):
              print("GPU memory insufficient for prediction. Terminating runtime to save compute units, restart with TPU.")
              runtime.unassign()
            else: raise
          # Reconstruct full tile from valid indices (C-order)
          if raster_precision == 0:
            prediction_flat = np.full(n_pixels, nodatavalue, dtype=np.int16)
            prediction_flat[valid_indices] = np.round(prediction).astype(np.int16)
          else:
            prediction_flat = np.full(n_pixels, nodatavalue, dtype=np.float32)
            prediction_flat[valid_indices] = np.round(prediction, raster_precision)
          prediction = None
          prediction_tile = prediction_flat.reshape((template_tile_y, template_tile_x), order='C')
          prediction_flat = None
          export_array_as_tif(prediction_tile, iteration_tile_path, template=template_tile_path, compress=False)
          prediction_tile = None

          tile_iteration_index += 1
          iteration_progress_label.value = f"Iteration progress: {tile_iteration_index} / {n_iterations_for_tile}"

        # Release tile resources
        feature_stack = valid_indices = None
        tile_progress_label.value = f"Tile progress: {tile_count} / {n_stacks}"

      # Merge tiles for each iteration
      tile_progress_label.value = "Tile progress: merging"
      for model_iteration in range(1, scenario_iterations + 1):
        prediction_iteration_filename = f"{scenario}__{selected_model}_iteration_{model_iteration}.tif"
        prediction_iteration_path = join(iterations_dir, prediction_iteration_filename)
        tile_cache_iteration_dir = tile_cache_dirs[model_iteration]
        if not exists(prediction_iteration_path):
          prediction_array = np.empty((0, template_tile_x))
          tile_files = sorted([f for f in os.listdir(tile_cache_iteration_dir) if f.endswith('.tif')],
                              key=lambda x: int(x.split('_')[-1].split('.')[0]))
          for tile_file in tile_files:
            tile_path = join(tile_cache_iteration_dir, tile_file)
            tile_array = read_raster_as_array(tile_path)
            prediction_array = np.vstack((prediction_array, tile_array))
          export_array_as_tif(prediction_array, prediction_iteration_path, template=template_base_path, compress=True)
          prediction_array = None
        # Delete tile cache for this iteration
        if exists(tile_cache_iteration_dir):
          shutil.rmtree(tile_cache_iteration_dir)

        iteration_progress_label.value = f"Merge progress: {model_iteration} / {scenario_iterations}"

  else:
    tile_progress_label.value = "Tile progress: complete"
    iteration_progress_label.value = "Iteration progress: complete"

  scenario_progress_label.value = f"Scenario progress: {scenario_index + 1} / {n_scenarios}"

print("\nScenario iterations complete.")

In [ ]:
# Merge oldgrowth iteration pairs
# Old-growth scenarios exist in two versions: _1 uses land-use proxy for pre-Landsat
# undisturbed forest (e.g. PAs without history of exploitation) and removes all Landsat-derived
# disturbance. _2 removes all Landsat-derived disturbance. The merge takes the maximum mean,
# avoiding underestimation where proxy may not capture all oldgrowth characteristics of the
# entire project area. Merging at iteration level (rather than of the calculated mean)
# allows Monte Carlo uncertainty propagation when calculating AGBD percentage loss between
# the old-growth alternative scenario baseline to the 'actual' state.

# Identify oldgrowth _1 iteration directories
oldgrowth_v1_dirs = [d for d in os.listdir(scenario_iterations_dir)
                     if 'oldgrowth' in d and d.endswith('_1_iterations')]

if not oldgrowth_v1_dirs: print("No oldgrowth version 1 iteration directories found to merge.")
else:
    n_oldgrowth = len(oldgrowth_v1_dirs)
    oldgrowth_progress_label = widgets.Label(value=f"Oldgrowth scenario progress: 0 / {n_oldgrowth}")
    iteration_merge_progress_label = widgets.Label(value="Iteration merge progress: -")
    display(oldgrowth_progress_label, iteration_merge_progress_label)

    for og_index, v1_dir_name in enumerate(oldgrowth_v1_dirs):
        # Define directory names and paths
        v2_dir_name = v1_dir_name.replace('_1_iterations', '_2_iterations')
        merged_dir_name = v1_dir_name.replace('_1_iterations', '_iterations')
        v1_dir_path = join(scenario_iterations_dir, v1_dir_name)
        v2_dir_path = join(scenario_iterations_dir, v2_dir_name)
        merged_dir_path = join(scenario_iterations_dir, merged_dir_name)

        # Skip if merged directory already exists with iterations
        if exists(merged_dir_path):
            existing_merged = [f for f in os.listdir(merged_dir_path) if f.endswith('.tif')]
            if existing_merged:
                iteration_merge_progress_label.value = f"Iteration merge progress: skipped (exists)"
                oldgrowth_progress_label.value = f"Oldgrowth scenario progress: {og_index + 1} / {n_oldgrowth}"
                continue

        # Check version 2 exists
        if not exists(v2_dir_path):
            print(f"Warning: {v2_dir_name} not found, skipping merge for {v1_dir_name}")
            oldgrowth_progress_label.value = f"Oldgrowth scenario progress: {og_index + 1} / {n_oldgrowth}"
            continue

        # Collect iteration files from both versions
        v1_files = sorted([f for f in os.listdir(v1_dir_path) if f.endswith('.tif')])
        v2_files = sorted([f for f in os.listdir(v2_dir_path) if f.endswith('.tif')])
        if len(v1_files) != len(v2_files):
            print(f"Warning: iteration count mismatch ({len(v1_files)} vs {len(v2_files)}), skipping {v1_dir_name}")
            oldgrowth_progress_label.value = f"Oldgrowth scenario progress: {og_index + 1} / {n_oldgrowth}"
            continue
        makedirs(merged_dir_path, exist_ok=True)
        n_iterations = len(v1_files)

        # Merge each iteration pair
        for iter_index, v1_file in enumerate(v1_files):
            # Construct merged filename (remove _1 suffix from scenario name in filename)
            merged_file = v1_file.replace('_1__', '__')
            merged_path = join(merged_dir_path, merged_file)
            if not exists(merged_path):
                # Construct v2 filename and paths
                v2_file = v1_file.replace('_1__', '_2__')
                v1_path = join(v1_dir_path, v1_file)
                v2_path = join(v2_dir_path, v2_file)
                if not exists(v2_path):
                    print(f"Warning: {v2_file} not found, skipping")
                    continue

                # Load both iterations
                v1_array = read_raster_as_array(v1_path)
                v2_array = read_raster_as_array(v2_path)

                # Take maximum value per pixel (nodata remains nodata as both versions share same mask)
                merged_array = np.maximum(v1_array, v2_array)

                # Export merged iteration
                export_array_as_tif(merged_array, merged_path, template=v1_path, compress=True)
                merged_array = v1_array = v2_array = None

            iteration_merge_progress_label.value = f"Iteration merge progress: {iter_index + 1} / {n_iterations}"
        oldgrowth_progress_label.value = f"Oldgrowth scenario progress: {og_index + 1} / {n_oldgrowth}"

    print("Oldgrowth iteration merging complete.")

# Predictions with uncertainty


In [ ]:
# Collect scenarios with iterations
# Excludes oldgrowth _1 and _2 variants since merged versions are used for statistics
scenarios_iterations_list = []
for subdir in os.listdir(scenario_iterations_dir):
    scenario_name = subdir[:-11]
    if 'oldgrowth' in scenario_name and (scenario_name.endswith('_1') or scenario_name.endswith('_2')):
        continue
    scenarios_iterations_list.append(scenario_name)

# Select scenarios to calculate mean, confidence intervals and uncertainty
print("scenarios_to_calculate = [")
for scenario in sorted(scenarios_iterations_list):
    print(f'  "{scenario}",')
print("]")

In [ ]:
scenarios_to_calculate = [
  "2018",
  "2019",
  "2020",
  "2021",
  "2021_undisturbed_since_1993",
  "2021_undisturbed_since_oldgrowth",
  "2021_oldgrowth_recovery",
  "2022",
  "2023",
  "2024",
  "2024_undisturbed_since_1996",
  "2024_undisturbed_since_1997",
  "2024_undisturbed_since_1998",
  "2024_undisturbed_since_1999",
  "2024_undisturbed_since_2000",
  "2024_undisturbed_since_2001",
  "2024_undisturbed_since_2002",
  "2024_undisturbed_since_2003",
  "2024_undisturbed_since_2004",
  "2024_undisturbed_since_2005",
  "2024_undisturbed_since_2006",
  "2024_undisturbed_since_2007",
  "2024_undisturbed_since_2008",
  "2024_undisturbed_since_2009",
  "2024_undisturbed_since_2010",
  "2024_undisturbed_since_2011",
  "2024_undisturbed_since_2012",
  "2024_undisturbed_since_2013",
  "2024_undisturbed_since_2014",
  "2024_undisturbed_since_2015",
  "2024_undisturbed_since_2016",
  "2024_undisturbed_since_2017",
  "2024_undisturbed_since_2018",
  "2024_undisturbed_since_2019",
  "2024_undisturbed_since_2020",
  "2024_undisturbed_since_2021",
  "2024_undisturbed_since_2022",
  "2024_undisturbed_since_2023",
  "2024_undisturbed_since_2024",
  "2024_undisturbed_since_oldgrowth",
  "2024_oldgrowth_recovery",
  "2024_road_mat_daling_deforestation_2023_30m_degradation_buffer",
]

In [ ]:
# Check iteration quality
# High proportion of values <= 0 may indicate corrupt iterations.
# Delete and repredict affected files if necessary.
check_iterations = False
nonpositive_threshold_percent = 1

# Check the number of prediction iterations
scenario_iterations = {}
for scenario in scenarios_to_calculate:
    iterations_dir = join(scenario_iterations_dir, f"{scenario}_iterations")
    iterations = 0
    for subdir in os.listdir(iterations_dir):
        if subdir.endswith(".tif"):
            if check_iterations:
                iteration_path = join(iterations_dir, subdir)
                iteration_array = read_raster_as_array(iteration_path)
                valid_mask = iteration_array != nodatavalue
                n_valid = np.count_nonzero(valid_mask)
                if n_valid > 0:
                    nonpositive_count = np.count_nonzero(iteration_array[valid_mask] <= 0)
                    nonpositive_percent = (nonpositive_count / n_valid) * 100
                    if nonpositive_percent > nonpositive_threshold_percent:
                        print(f"Warning: {subdir} has {nonpositive_percent:.1f}% values <= 0 in valid pixels.")
            iterations += 1
    scenario_iterations[scenario] = iterations
    print(f"There are {iterations} prediction iterations for scenario {scenario} statistics.")

In [ ]:
# Raster precision settings
mean_precision = 2
ci_precision = 2
uncertainty_precision = 2

# Quantify prediction uncertainty using Monte Carlo simulation with percentile-based
# confidence intervals (IPCC Approach 2).

# Each model iteration sampled target values from normal distributions using GEDI L4A
# agbd (mean) and agbd_se (standard deviation). The resulting collection of respective
# prediction iterations represent the distribution of possible AGBD values
# given measurement uncertainty.

# Confidence intervals derive from percentiles of the empirical (actual rather
# than theoretical) distribution: CI half-width = (P_97.5 - P_2.5) / 2
# Percentiles directly characterise the distribution without assuming normality,
# are robust to outliers, and handle bounded distributions (AGBD ≥ 0).

# Percentage uncertainty = CI half-width / mean * 100
# This represents precision of estimates given input uncertainty, not total predictive
# accuracy (Liang et al. 2023, p.6).

# References:
# IPCC (2006) Guidelines Vol.1 Ch.3: Uncertainties, Section 3.2.3.2
# IPCC (2019) Refinement Vol.1 Ch.3: Uncertainties, Section 3.2.3.2
# Liang et al. (2023) Remote Sensing of Environment 284:113367

# Progress labels
scenario_progress_index = 0
scenario_progress_label = widgets.Label(f"Scenario stats progress: {scenario_progress_index}/{len(scenarios_to_calculate)}")
iteration_progress_label = widgets.Label(value="Iteration loading progress: -")
display(scenario_progress_label, iteration_progress_label)

# Loop through the scenarios
for scenario, iteration_count in scenario_iterations.items():
    base_filename = f"{scenario}__{selected_model}"
    iterations_dir = join(scenario_iterations_dir, f"{scenario}_iterations")

    # Define output paths
    mean_filename = f"mean__{base_filename}.tif"
    mean_path = join(predictions_dir, mean_filename)
    ci_filename = f"ci_95__{base_filename}.tif"
    ci_path = join(predictions_dir, ci_filename)
    uncertainty_filename = f"uncertainty__{base_filename}.tif"
    uncertainty_path = join(predictions_dir, uncertainty_filename)

    # If any output does not exist, recalculate all
    if not exists(mean_path) or not exists(ci_path) or not exists(uncertainty_path):

        # Collect iteration file paths
        iteration_paths = [join(iterations_dir, f) for f in os.listdir(iterations_dir) if f.endswith(".tif")]
        n_iterations = len(iteration_paths)
        iteration_progress_label.value = f"Iteration loading progress: 0 / {n_iterations}"

        # Load iterations sequentially, building stack incrementally
        for iteration_index, iteration_path in enumerate(iteration_paths):
            iteration_array = read_raster_as_array(iteration_path)
            if iteration_index == 0:
                iteration_stack = np.empty((n_iterations, *iteration_array.shape), dtype=np.float64)
                valid_mask = iteration_array != nodatavalue
                template_path = iteration_path
            iteration_stack[iteration_index] = np.where(iteration_array == nodatavalue, np.nan, iteration_array)
            iteration_progress_label.value = f"Iteration loading progress: {iteration_index + 1} / {n_iterations}"

        # Suppress warnings for all-NaN slices (expected for nodata pixels, handled by valid_mask)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", RuntimeWarning)

            # Calculate mean across iterations
            mean_array = np.nanmean(iteration_stack, axis=0)

            # Sort once, index at percentile positions. Faster than np.nanpercentile.
            # Valid because nodata mask is identical across iterations.
            sorted_stack = np.sort(iteration_stack, axis=0)
            iteration_stack = None

            n = sorted_stack.shape[0]
            lower_index = 0.025 * (n - 1)
            upper_index = 0.975 * (n - 1)
            lower_floor = int(np.floor(lower_index))
            upper_floor = int(np.floor(upper_index))
            lower_fraction = lower_index - lower_floor
            upper_fraction = upper_index - upper_floor

            # Interpolate between adjacent sorted values
            ci_lower = (sorted_stack[lower_floor] * (1 - lower_fraction) +
                        sorted_stack[min(lower_floor + 1, n - 1)] * lower_fraction)
            ci_upper = (sorted_stack[upper_floor] * (1 - upper_fraction) +
                        sorted_stack[min(upper_floor + 1, n - 1)] * upper_fraction)
            sorted_stack = None

            # CI half-width
            ci_halfwidth = (ci_upper - ci_lower) / 2

            # Relative uncertainty as percentage
            percentage_uncertainty = np.zeros_like(mean_array, dtype=np.float64)
            nonzero_mean_mask = valid_mask & (mean_array != 0)
            percentage_uncertainty[nonzero_mean_mask] = (ci_halfwidth[nonzero_mean_mask] / np.abs(mean_array[nonzero_mean_mask])) * 100

            # Round and apply nodata mask
            if mean_precision == 0:
                mean_rounded = np.round(mean_array, mean_precision).astype(np.int16)
                mean_rounded = np.where(valid_mask, mean_rounded, nodatavalue).astype(np.int16)
            else:
                mean_rounded = np.round(mean_array, mean_precision)
                mean_rounded = np.where(valid_mask, mean_rounded, nodatavalue)
            if ci_precision == 0:
                ci_rounded = np.round(ci_halfwidth, ci_precision).astype(np.int16)
                ci_rounded = np.where(valid_mask, ci_rounded, nodatavalue).astype(np.int16)
            else:
                ci_rounded = np.round(ci_halfwidth, ci_precision)
                ci_rounded = np.where(valid_mask, ci_rounded, nodatavalue)
            if uncertainty_precision == 0:
                uncertainty_rounded = np.round(percentage_uncertainty, uncertainty_precision).astype(np.int16)
                uncertainty_rounded = np.where(valid_mask, uncertainty_rounded, nodatavalue).astype(np.int16)
            else:
                uncertainty_rounded = np.round(percentage_uncertainty, uncertainty_precision)
                uncertainty_rounded = np.where(valid_mask, uncertainty_rounded, nodatavalue)

            # Export statistics arrays as rasters
            export_array_as_tif(mean_rounded, mean_path, template=template_path)
            export_array_as_tif(ci_rounded, ci_path, template=template_path)
            export_array_as_tif(uncertainty_rounded, uncertainty_path, template=template_path)

        # Release memory
        iteration_stack = sorted_stack = None

    else:
        iteration_progress_label.value = "Iteration loading progress: skipped (exists)"
        print(f"{mean_filename}, {ci_filename} and {uncertainty_filename} already exist.")

    scenario_progress_index += 1
    scenario_progress_label.value = f"Scenario stats progress: {scenario_progress_index}/{len(scenarios_to_calculate)}"

print("\nStatistics calculations complete.")

mean__2024_oldgrowth_recovery__agbd_251203_161707.tif, ci_95__2024_oldgrowth_recovery__agbd_251203_161707.tif and uncertainty__2024_oldgrowth_recovery__agbd_251203_161707.tif already exist.
mean__2024_road_mat_daling_deforestation_2023_30m_degradation_buffer__agbd_251203_161707.tif, ci_95__2024_road_mat_daling_deforestation_2023_30m_degradation_buffer__agbd_251203_161707.tif and uncertainty__2024_road_mat_daling_deforestation_2023_30m_degradation_buffer__agbd_251203_161707.tif already exist.

Statistics calculations complete.


# Disconnect runtime

In [ ]:
# Useful for stopping background execution upon completion
runtime.unassign()